Runtime 20 - 25 mins


---
change the stock market code down bellow (stock_code)


---
Clck on Runtime -> Run all 




In [1]:
#@title Install Dependencies
!pip install -q yfinance
!pip install pandas_datareader -q
!pip install -q plotly -q
!pip install nsepy -q
!pip install tensorflow

In [2]:
#@title Import dependencies
import pandas as pd
import numpy as np 
import datetime
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from nsepy import get_history as gh
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
import datetime as dt

Populating the interactive namespace from numpy and matplotlib


Write the stock market code here with the start and end date

In [3]:
stock_code = 'INFY'
start = dt.datetime(2013,1,1)     #Year,Month,Day
lst = list(map(int , datetime.datetime.now().strftime("%Y-%m-%d").split("-")))
end   = dt.datetime(2020,12,31)   #Year,Month,Day

In [4]:
#@title Download the stock data
stk_data = gh(symbol=stock_code,start=start,end=end)

In [5]:
#@title Show the downloaded data
stk_data

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2013-01-01,INFY,EQ,2318.70,2327.60,2329.90,2305.55,2307.10,2308.85,2318.02,369999,8.576666e+13,14479,195016,0.5271
2013-01-02,INFY,EQ,2308.85,2316.20,2328.00,2303.50,2309.20,2309.20,2315.36,543743,1.258958e+14,45625,321559,0.5914
2013-01-03,INFY,EQ,2309.20,2315.00,2348.85,2311.00,2346.95,2337.90,2329.97,1324911,3.087005e+14,60958,963493,0.7272
2013-01-04,INFY,EQ,2337.90,2348.40,2354.95,2332.00,2350.00,2349.55,2345.09,1056751,2.478176e+14,55290,710935,0.6728
2013-01-07,INFY,EQ,2349.55,2351.10,2379.90,2335.55,2373.30,2374.80,2367.82,1095203,2.593243e+14,52530,796335,0.7271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,INFY,EQ,1253.05,1249.90,1249.90,1226.00,1237.35,1236.05,1236.34,7313885,9.042416e+14,172066,2637421,0.3606
2020-12-28,INFY,EQ,1236.05,1238.45,1248.00,1236.00,1239.45,1240.30,1240.58,4607051,5.715406e+14,123013,2098658,0.4555
2020-12-29,INFY,EQ,1240.30,1235.00,1254.45,1235.00,1249.85,1250.30,1246.33,6878105,8.572420e+14,132402,3219736,0.4681


In [6]:
#@title Prepare Data for model training

stk_data['Date'] = stk_data.index
data2 = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close'])
data2['Date'] = stk_data['Date']
data2['Open'] = stk_data['Open']
data2['High'] = stk_data['High']
data2['Low'] = stk_data['Low']
data2['Close'] = stk_data['Close']


train_set = data2.iloc[:, 1:2].values
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train_set)
X_train = []
y_train = []
for i in range(60, len(stk_data)-1):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0]) 
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [7]:
#@title Train the model
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 100))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 100, batch_size = 64)

Epoch 1/100
30/30 [==============================] - 15s 228ms/step - loss: 0.0208
Epoch 2/100
30/30 [==============================] - 7s 251ms/step - loss: 0.0039
Epoch 3/100
30/30 [==============================] - 7s 246ms/step - loss: 0.0035
Epoch 4/100
14/30 [=============>................] - ETA: 4s - loss: 0.0032

In [ ]:
#@title See how the model performs on 2021 years data
testdataframe= gh(symbol=stock_code,start=dt.datetime(2021,1,1),end=dt.datetime(2021,12,31))
testdataframe['Date'] = testdataframe.index
testdata = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close'])
testdata['Date'] = testdataframe['Date']
testdata['Open'] = testdataframe['Open']
testdata['High'] = testdataframe['High']
testdata['Low'] = testdataframe['Low']
testdata['Close'] = testdataframe['Close']
real_stock_price = testdata.iloc[:, 1:2].values
dataset_total = pd.concat((data2['Open'], testdata['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(testdata) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)


df = pd.DataFrame()
df['Date'] = testdataframe['Date']
df['Open'] = testdataframe['Open']
df["Predicted"] = predicted_stock_price
import plotly.express as px
fig = px.line(df, x="Date", y="Predicted", markers=True)
fig.add_trace(go.Scatter(x=df["Date"], y=df["Open"],
                    mode='lines+markers',
                    name='Actual'))


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Date=%{x}<br>Predicted=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.date(2021, 1, 1), datetime.date(2021, 1, 4),
                          datetime.date(2021, 1, 5), ..., datetime.date(2022, 2, 4),
                          datetime.date(2022, 2, 7), datetime.date(2022, 2, 8)], dtype=object),
              'xaxis': 'x',
              'y': array([1239.7596, 1241.8396, 1246.431 , ..., 1752.424 , 1756.8464, 1744.4144],
                         dtype=float32),
              'yaxis': 'y'},
             {'mode': 'lines+markers',
              'name': 'Actual',
              'type': 'scatter',
              'x': array([datetime.date(2021, 1, 1), datetime.date(2021, 1, 4),
                          datetime.date(2021, 1, 5), ..., datetime.date(2022, 2, 4),
                          datetime.date(2022, 2, 7), datetime.date(2022, 2, 8)], dtype=object),
              'y': array([1257.9, 1269. , 1282. , ..., 1732. , 1722.2, 1710. ])}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Predicted'}}}
})

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df["Open"],df["Predicted"])

In [ ]:
#@title  Make Predictions 
from numpy import array
from datetime import relativedelta
tod = dt.datetime.now()
lst = list(map(int , tod.strftime("%Y-%m-%d").split("-")))
testdataframe= gh(symbol=stock_code,start=dt.datetime(2021,1,1),end=dt.datetime(lst[0],lst[1],lst[2]))


temp = sc.transform(testdataframe.iloc[-60:,3:4].values)

x_input=temp.reshape(1,-1)
x_input.shape


temp_input=list(x_input)
temp_input=temp_input[0].tolist()
lst_output=[]
n_steps=60
i=0
while(i<730):
    if(len(temp_input)>60):
        x_input=np.array(temp_input[1:])
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = regressor.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = regressor.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        lst_output.extend(yhat.tolist())
        i=i+1

preds = np.array(lst_output)
preds = sc.inverse_transform(preds)
from numpy.lib.function_base import kaiser
t = 730
lst = []
k = dt.datetime.now()

while(t>0):
  temp = k + relativedelta(days = 1)
  k = temp 
  lst.append(temp.strftime("%Y-%m-%d"))
  t-=1
df1 = pd.DataFrame()
df1['Date'] = lst
df1["Predicted"] = preds
import plotly.express as px
fig = px.line(df1, x="Date", y="Predicted", markers=True)
fig.show()  


ValueError: cannot reshape array of size 333 into shape (1,60,1)

In [ ]:
import plotly.express as px
fig = px.line(df1, x="Date", y="Predicted", markers=True)
fig.add_trace(go.Scatter(x=df["Date"], y=df["Open"],
                    mode='lines+markers',
                    name='Actual'))


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Date=%{x}<br>Predicted=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2022-02-09', '2022-02-10', '2022-02-11', ..., '2024-02-06',
                          '2024-02-07', '2024-02-08'], dtype=object),
              'xaxis': 'x',
              'y': array([1724.85500437, 1711.27554232, 1702.48663583, ..., 1095.80310101,
                          1095.80292766, 1095.80258096]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Predicted'}}}
})

In [ ]:
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Date=%{x}<br>Predicted=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2022-02-09', '2022-02-10', '2022-02-11', ..., '2024-02-06',
                          '2024-02-07', '2024-02-08'], dtype=object),
              'xaxis': 'x',
              'y': array([3853.39531898, 3939.34789658, 4057.56272316, ..., 2686.95702076,
                          2691.08559608, 2696.78781271]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Predicted'}}}
})

In [ ]:
len(df)

274